<a href="https://colab.research.google.com/github/takuminmin-m/hisseki_test_py/blob/main/hisseki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ランタイムの接続が切れないようにするJavaScriptコード
DevToolsのコンソールに入力

JavaScript code to stop for the runtime to disconnect 

```javascript
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click();
}
setInterval(KeepClicking,60000);
```



# データのインポート import data
google driveをマウント

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ライブラリの読み込みなど

Load libraries and more

In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

import os
import numpy as np
import IPython.display as display
import random
import matplotlib.pyplot as plt
import pathlib

AUTOTUNE = tf.data.experimental.AUTOTUNE

作業ディレクトリの移動とデータの読み込み

Change the working directory and load labels.txt and handwriting images

In [ ]:
%cd /content/drive/MyDrive/colab/hisseki_test_py

def preprocess_image(image):
  image_4ch = tf.image.decode_image(image, channels=4, expand_animations=False)
  image_4ch = tf.image.resize(image_4ch, [128, 128])
  # image = tf.cast(tf.reduce_sum(image_4ch, 2, keepdims=True), tf.float32)
  # image /= 255.0  # normalize to [0,1] range
  image_4ch /= 255.0

  return image_4ch

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

label_file = open("tf_datas/labels.txt")
labels_str = label_file.readlines()
label_file.close()
all_labels = list(map(lambda str: int(str), labels_str))

data_num = len(all_labels)
member_num = len(set(all_labels))
image_paths = []
for i in range(data_num):
  image_paths.append("/content/drive/MyDrive/colab/hisseki_test_py/tf_datas/hisseki%d.png" % i)

image_paths = tf.data.Dataset.from_tensor_slices(image_paths)
all_images = list(image_paths.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE))
all_datas = []
for i in range(data_num):
  all_datas.append({"image": all_images[i], "label": all_labels[i]})

# ランダムに画像をテスト表示
# randomly choose a handwriting image and show to test
plt.imshow(random.choice(all_images))
plt.grid(False)
plt.show()

教師データの前処理

Preprocess training data

In [ ]:
def split(list, slice_index):
  shuffled_list = list
  random.shuffle(shuffled_list)
  return shuffled_list[:slice_index], shuffled_list[slice_index:]

def separate_image_and_label(list):
  images = []
  labels = []
  for i in range(len(list)):
    images.append(list[i]["image"])
    labels.append(list[i]["label"])
  return images, labels

random.shuffle(all_datas)
slice_index = int(data_num * 0.9)
train_datas, test_datas = split(all_datas, slice_index)

train_images, train_labels = separate_image_and_label(train_datas)
test_images, test_labels = separate_image_and_label(test_datas)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# モデルを作成 Make a model

モデルの設定

Set model

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 4)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(member_num, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

keras.utils.plot_model(model, "model_plot.png", show_shapes=True)

# 訓練 Train

In [ ]:
model.fit(train_images, train_labels, epochs=20)

評価

Evaluation

In [ ]:
model.evaluate(test_images, test_labels, verbose=2)